# Plot groups' analysis
## Set parameters

In [ ]:
# ####################################### MAC DIRECTORIES #######################################
data_root = "/Volumes/Ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
plots_root = "/Volumes/Ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/results/experiment/plots/"

In [ ]:
# ###################################### GNOME DIRECTORIES ######################################
data_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
plots_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/results/experiment/plots/"

In [ ]:
# ###################################### LOCAL DIRECTORIES ######################################
data_root = "./data/experiments/soumnya/experiment/"
plots_root = "./plots/soumnya/experiment/"

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
branches_to_exclude = ["retina", "VS", "grv", "fiber tracts", "CB"]
normalization = "Density"                           # call get_normalization_methods() on a AnimalGroup object to know its available normalization methods
regions_to_plot_selection_method = "pls"             # Available options are: "summary structures", "major divisions" "depth <n>", "pls", "structural level <n>"
                                                    # where <n> is an integer of the depth/level desired
regions_to_plot_pls_csv = f"{data_root}../experiment/BraiAn_output/CF-SF/pls_cfos_density_salient_regions_above_1_2.csv"  # used only if the method for selecting the regions is "pls"
min_animals_cross_correlation = 5                   # 'None' means that ALL the animals must have the region, otherwise the correlation is NaN
plot_regions_with_insufficient_data = True          # what to do with brain regions with less animals than min_animals_cross_correlation
saved_plot_extension = ".html"                      # '.html' for interactive plot
                                                    # '.svg' for vectorized image
                                                    # '.png'/'.jpg'/... for rasterized image


# ########################################## BAR PLOT ###########################################
bar_height = 10_000
bar_width = 1_500
bar_use_acronyms = False
bar_save_plot = True
bar_show_plot = False

# ###################################### CORRELATION MATRIX #####################################
matrix_cell_height = 5
matrix_cell_ratio = 3/2
matrix_min_plot_height = 500
matrix_save_plot = False
matrix_show_plot = False

# ######################################## CHORD DIAGRAM ########################################
chord_p_cutoff = 0.05                              # 1 if you don't want to filter by p-value
chord_r_cutoff = 0.8
chord_plot_size = 1200
chord_no_background = False
chord_regions_size = 10
chord_regions_font_size = 10
chord_max_edge_width = 5
chord_use_weighted_edge_widths = True
chord_use_colorscale_edges = True
chord_save_plot = False
chord_show_plot = False
chord_bottom_annotations = dict(
    annotation1 = "Dark grey nodes are regions with insufficient data to compute cross correlation",
    annotation2 = "Light grey nodes are regions with no correlation with others above the threshold",
    annotation3 = "This is the third annotation",
    # howmany annotations desired with the following format:
    # annotations<k> = "<annotation>"
)

In [ ]:
# SOUMNYA FEMALES+MALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control",
    "Stress"
]
plots_output_folder = "C-S"

In [ ]:
# SOUMNYA ALL - 2 Groups {Stress|Control} + 2 Groups {Males|Females}
experiment = "soumnya"
groups = [
    "Control (Females)",
    "Stress (Females)",
    "Control (Males)",
    "Stress (Males)"
]
plots_output_folder = "CF-SF-CM-SM"

In [ ]:
# SOUMNYA FEMALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control (Females)",
    "Stress (Females)",
]
plots_output_folder = "CF-SF"

In [ ]:
# SOUMNYA MALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control (Males)",
    "Stress (Males)"
]
plots_output_folder = "CM-SM"

In [ ]:
# SHILA - 3 Groups {Control|Stress|Resilient}
experiment = "shila"
groups = [
    "Control",
    "Stress",
    "Resilient"
]
plots_output_folder = "C-S-R"

In [ ]:
# SHILA - 2 Groups {Control|Stress+Resilient}
experiment = "shila"
groups = [
    "Control",
    "Stress"
]
plots_output_folder = "C-S"

In [ ]:
# TEST DATA - Sample data
experiment = "test"
groups = [
    "Control",
    "Stress",
    "Resilient"
]
plots_output_folder = "C-S-R"

## Script's code
run all cell below

In [ ]:
import BraiAn
import os

data_input_path = os.path.join(data_root, "BraiAn_output")
plots_output_path = os.path.join(plots_root, plots_output_folder)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
animal_groups = []
for group in groups:
    animal_group = BraiAn.AnimalGroup.from_csv(group, data_input_path, f"cell_counts_{group}.csv")
    animal_groups.append(animal_group)
    print(f"Group '{animal_group.name}' - #animals: {animal_group.n}, marker: {animal_group.marker}")


In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

In [ ]:
match regions_to_plot_selection_method:
    case "summary structures":
        # selects the Summary Strucutures
        AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case "pls":
        AllenBrain.select_from_csv(regions_to_plot_pls_csv, key="acronym")
    case s if s.startswith("depth"):
        n = regions_to_plot_selection_method.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'regions_to_plot_selection_method'")
        AllenBrain.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = regions_to_plot_selection_method.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'regions_to_plot_selection_method'")
        AllenBrain.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{regions_to_plot_selection_method}' for regions_to_plot_selection_method")
regions_to_plot = AllenBrain.get_selected_regions()
print(f"You selected {len(regions_to_plot)} regions to plot.")

In [ ]:
fig = BraiAn.plot_groups(normalization, AllenBrain, *animal_groups, selected_regions=regions_to_plot,
                            use_acronyms=bar_use_acronyms, height=bar_height, width=bar_width, bargap=0.3, bargroupgap=0.0)
regions_to_plot_selection_method_str = regions_to_plot_selection_method.replace(" ", "_")

if bar_save_plot:
    plot_filename = f"barplot_{plots_output_folder}_{normalization}_{animal_groups[0].marker}_{regions_to_plot_selection_method_str}{saved_plot_extension}".lower()
    plot_filepath = os.path.join(plots_output_path, plot_filename)
    match saved_plot_extension.lower():
        case ".html":
            fig.write_html(plot_filepath)
        case _:
            fig.write_image(plot_filepath)
if bar_show_plot:
    fig.show()

In [ ]:
if matrix_save_plot or matrix_show_plot or chord_save_plot or chord_show_plot:
    groups_cross_correlations = []
    for group in animal_groups:
        r, p = group.cross_correlation(normalization, regions_to_plot, min_animals=min_animals_cross_correlation)
        if not plot_regions_with_insufficient_data:
            r.dropna(axis=0, how="all", inplace=True)
            r.dropna(axis=1, how="all", inplace=True)
            p.dropna(axis=0, how="all", inplace=True)
            p.dropna(axis=1, how="all", inplace=True)
        groups_cross_correlations.append((r, p))

In [ ]:
if matrix_save_plot or matrix_show_plot:
    for group, (r,p) in zip(animal_groups, groups_cross_correlations):
        title = f"{group.name} Pearson cross correlation matrix (n = {group.n})"
        fig = BraiAn.plot_cross_correlation(r=r, p=p,
                title=title,
                cell_height=matrix_cell_height, min_plot_height=matrix_min_plot_height,
                aspect_ratio=matrix_cell_ratio)
        if matrix_save_plot:
            plot_filename = f"correlation_matrix_{group.name}_{normalization}_{group.marker}_{regions_to_plot_selection_method_str}{saved_plot_extension}".lower()
            plot_filepath = os.path.join(plots_output_path, plot_filename)
            match saved_plot_extension.lower():
                case ".html":
                    fig.write_html(plot_filepath)
                case _:
                    fig.write_image(plot_filepath)
        if matrix_show_plot:
            fig.show()

In [ ]:
if chord_save_plot or chord_show_plot:
    for group, (r,p) in zip(animal_groups, groups_cross_correlations):
        group_annotations = dict(
                                subtitle = "",
                                **chord_bottom_annotations
                            )
        fig = BraiAn.draw_chord_plot(r=r, p=p, r_cutoff=chord_r_cutoff, p_cutoff=chord_p_cutoff,
                                    AllenBrain=AllenBrain,
                                    ideograms_a=50,
                                    title=f"{group.name} connectomics graph from Pearson correlation (n = {group.n}, |r| >= {chord_r_cutoff}, p <= {chord_p_cutoff})",
                                    size=chord_plot_size,
                                    no_background=chord_no_background,
                                    regions_size=chord_regions_size,
                                    regions_font_size=chord_regions_font_size,
                                    max_edge_width=chord_max_edge_width,
                                    use_weighted_edge_widths=chord_use_weighted_edge_widths,
                                    colorscale_edges=chord_use_colorscale_edges,
                                    **group_annotations
        )
        if chord_save_plot:
            plot_filename = f"chord_plot_{group.name}_{normalization}_{group.marker}_{regions_to_plot_selection_method_str}{saved_plot_extension}".lower()
            plot_filepath = os.path.join(plots_output_path, plot_filename)
            match saved_plot_extension.lower():
                case ".html":
                    fig.write_html(plot_filepath)
                case _:
                    fig.write_image(plot_filepath)
        if chord_show_plot:
            fig.show()

In [ ]:
import importlib
importlib.reload(BraiAn.plot)
importlib.reload(BraiAn.plot_chord)
importlib.reload(BraiAn)